In [1]:
import os
import re
import pandas as pd
import joblib
import PyPDF2
import pytesseract
from pdf2image import convert_from_path
from PIL import Image


In [2]:
KNOWN_PROBLEMATIC_FILES = {
    '5bccdc90-25cb-4304-9992-d279443adb0c.pdf',
    'b722da60-8b55-4545-b13a-07a4cad1cfbb.pdf', 
    '435175-128049-file0001.pdf',
    '125972-128068-file0001.pdf',
    '187429-128122-file0001.pdf',
    '436224-128050-file0001.pdf',
    '402914-128078-file0001.pdf',
    '152400-127701-file0001.pdf'
}

def extract_text_from_pdf(pdf_path):
    filename = os.path.basename(pdf_path)
    
    if filename in KNOWN_PROBLEMATIC_FILES:
        # Preprocesamiento seguro
        images = convert_from_path(pdf_path, dpi=100)
        all_text = []
        for image in images:
            width, height = image.size
            if width * height > 50_000_000:
                scale = (50_000_000 / (width * height)) ** 0.5
                image = image.resize((int(width * scale), int(height * scale)), Image.Resampling.LANCZOS)
            text = pytesseract.image_to_string(image, lang='spa', config='--psm 3')
            if text.strip():
                all_text.append(text.strip())
        return "\n".join(all_text)
    else:
        # Extracción estándar
        text = ""
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text.strip():
                    text += page_text + "\n"
        
        if text.strip():
            return text.strip()
        else:
            # OCR si no hay texto
            images = convert_from_path(pdf_path, dpi=300)
            ocr_text = []
            for image in images:
                page_text = pytesseract.image_to_string(image, lang='spa', config='--psm 3')
                if page_text.strip():
                    ocr_text.append(page_text.strip())
            return "\n".join(ocr_text)

def extract_text_from_image(filepath):
    img = Image.open(filepath)
    return pytesseract.image_to_string(img, lang='spa', config='--psm 3').strip()


In [3]:
document_types_and_paths = {
    'Carta de trabajo': '../data/Insumos/Carta de trabajo/',
    'Cedulas 2': '../data/Insumos/Cedulas 2/',
    'Pasaportes': '../data/Insumos/Pasaportes/',
    'Sustento de ingresos': '../data/Insumos/Sustento de ingresos/',
    'W-9': '../data/Insumos/W-9/'
}

def extract_corpus_from_documents(doc_paths):
    corpus = []
    total_processed = 0
    total_skipped = 0

    for doc_type, base_folder_path in doc_paths.items():
        if not os.path.exists(base_folder_path):
            continue

        for root, dirs, files in os.walk(base_folder_path):
            for filename in files:
                filepath = os.path.join(root, filename)
                file_extension = os.path.splitext(filename)[1].lower()
                
                try:
                    if file_extension == '.pdf':
                        extracted_text = extract_text_from_pdf(filepath)
                    elif file_extension in ['.png', '.jpg', '.jpeg', '.gif', '.tiff', '.bmp']:
                        extracted_text = extract_text_from_image(filepath)
                    else:
                        total_skipped += 1
                        continue

                    if extracted_text.strip():
                        corpus.append((extracted_text, doc_type))
                        total_processed += 1
                    else:
                        total_skipped += 1
                except:
                    total_skipped += 1
    
    return corpus, total_processed, total_skipped

In [4]:
corpus_file_path = '../data/processed_corpus.pkl'

if os.path.exists(corpus_file_path):
    df_corpus = joblib.load(corpus_file_path)
else:
    extracted_corpus_list, total_processed, total_skipped = extract_corpus_from_documents(document_types_and_paths)
    df_corpus = pd.DataFrame(extracted_corpus_list, columns=['documento_original', 'tipo'])
    
    if not df_corpus.empty:
        os.makedirs(os.path.dirname(corpus_file_path), exist_ok=True)
        joblib.dump(df_corpus, corpus_file_path)

# Estadísticas
print(f"Total documentos: {len(df_corpus)}")
print(f"Tipos: {df_corpus['tipo'].nunique()}")
print("\nDistribución:")
for doc_type, count in df_corpus['tipo'].value_counts().items():
    print(f"  {doc_type}: {count}")

df_corpus.head()


Total documentos: 344
Tipos: 5

Distribución:
  Carta de trabajo: 98
  Sustento de ingresos: 95
  Pasaportes: 91
  Cedulas 2: 41
  W-9: 19


,documento_original,tipo
0,o!\n\n(O) Tintorería Ecoloca\nN TACO\n\nPara: ...,Carta de trabajo
1,CICR\n\nCONSTANCIA DE TRABAJO\n\nA quien pueda...,Carta de trabajo
2,"Distribuidora Zoliannys, LLC.\n\nEIN 30-130830...",Carta de trabajo
3,CONSTANCIA\n\nAla atención de Banco Mercantil ...,Carta de trabajo
4,"Ave. Samuel Lewis y Calle 58, Obarrio , Torre ...",Carta de trabajo


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer # O PorterStemmer para stemming

# Descargar recursos de NLTK necesarios
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
    print("Descargado el recurso WordNet de NLTK (para lematización).")

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
    print("Descargadas las stopwords de NLTK.")

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    print("Descargado el tokenizador Punkt de NLTK.")

print("Configuración de NLTK completada.")


Descargado el recurso WordNet de NLTK (para lematización).
Configuración de NLTK completada.


[nltk_data] Downloading package wordnet to /Users/mb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# Función de preprocesamiento mejorada
from langdetect import detect
import spacy

# Función para detectar idioma
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'es'  # Default a español

# Función mejorada de preprocesamiento
def preprocess_text_improved(text, preserve_numbers=True):
    """
    Preprocesamiento mejorado que:
    - Detecta el idioma automáticamente
    - Usa stopwords apropiadas según el idioma
    - Preserva números importantes (opcional)
    - Maneja documentos muy cortos
    """
    if not isinstance(text, str) or len(text.strip()) < 10:
        return text if isinstance(text, str) else ""
    
    # Detectar idioma
    lang = detect_language(text)
    
    # Configurar stopwords según idioma detectado
    if lang == 'en':
        stop_words = set(stopwords.words('english'))
    else:  # Default español
        stop_words = set(stopwords.words('spanish'))
    
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar puntuación pero preservar algunos caracteres importantes
    text = text.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'))
    
    # Preservar o eliminar números según parámetro
    if not preserve_numbers:
        text = re.sub(r'\d+', '', text)
    else:
        # Preservar solo números que parecen importantes (fechas, códigos, etc.)
        # Mantener números de 4+ dígitos (años, códigos) y números con puntos/guiones
        text = re.sub(r'\b\d{1,3}\b(?!\d)', '', text)  # Eliminar números de 1-3 dígitos solos
    
    # Tokenización
    tokens = text.split()
    
    # Eliminar stop words y palabras muy cortas
    processed_tokens = []
    for word in tokens:
        if (len(word) > 2 and 
            word not in stop_words and 
            not word.isdigit() or len(word) > 3):  # Preservar números largos
            processed_tokens.append(word)
    
    return ' '.join(processed_tokens)

print("🔧 Función de preprocesamiento mejorada definida")
print("✅ Incluye detección de idioma y preservación selectiva de números")


🔧 Función de preprocesamiento mejorada definida
✅ Incluye detección de idioma y preservación selectiva de números


In [7]:
# 🔧 FUNCIÓN DE PREPROCESAMIENTO MEJORADA PARA CLASIFICACIÓN
import unicodedata

def preprocess_text_for_classification(text, preserve_domain_keywords=True):
    """
    Preprocesamiento optimizado para clasificación de documentos:
    - Normaliza acentos y caracteres especiales
    - Preserva palabras clave del dominio
    - Limpia ruido del OCR de manera inteligente
    - Maneja números de forma contextual
    """
    if not isinstance(text, str) or len(text.strip()) < 5:
        return text if isinstance(text, str) else ""
    
    # Detectar idioma
    lang = detect_language(text)
    
    # Configurar stopwords según idioma
    if lang == 'en':
        stop_words = set(stopwords.words('english'))
    else:
        stop_words = set(stopwords.words('spanish'))
    
    # Palabras clave importantes del dominio (no eliminar)
    domain_keywords = {
        'pasaporte', 'passport', 'cedula', 'cédula', 'carta', 'trabajo', 'constancia',
        'form', 'w9', 'taxpayer', 'sustento', 'ingresos', 'informe', 'contador',
        'republica', 'venezuela', 'bolivariana', 'director', 'ein', 'social', 'security'
    }
    
    # 1. Convertir a minúsculas
    text = text.lower()
    
    # 2. Normalizar acentos y caracteres especiales
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    
    # 3. Limpiar ruido común del OCR
    text = re.sub(r'[|\\/*+{}[\]()<>]', ' ', text)  # Caracteres extraños del OCR
    text = re.sub(r'[^\w\s.-]', ' ', text)  # Mantener solo letras, números, espacios, puntos y guiones
    text = re.sub(r'\s+', ' ', text)  # Múltiples espacios a uno solo
    
    # 4. Tokenización
    tokens = text.split()
    
    # 5. Procesamiento inteligente de tokens
    processed_tokens = []
    for token in tokens:
        # Saltar tokens muy cortos (menos de 2 caracteres)
        if len(token) < 2:
            continue
            
        # Preservar palabras clave del dominio
        if token in domain_keywords:
            processed_tokens.append(token)
            continue
            
        # Preservar números importantes (4+ dígitos, fechas, códigos)
        if re.match(r'^\d{4,}$|^\d{1,2}[-/]\d{1,2}[-/]\d{2,4}$|^v\d+$', token):
            processed_tokens.append(token)
            continue
            
        # Eliminar stopwords pero no palabras del dominio
        if token not in stop_words and len(token) > 2:
            processed_tokens.append(token)
    
    # 6. Limitar longitud final si es muy larga
    final_text = ' '.join(processed_tokens[:100])  # Máximo 100 tokens
    
    return final_text if final_text else text[:50]  # Fallback para textos muy problemáticos

print("🔧 Función de preprocesamiento optimizada para clasificación definida")
print("✅ Incluye normalización de acentos y preservación de palabras clave del dominio")


🔧 Función de preprocesamiento optimizada para clasificación definida
✅ Incluye normalización de acentos y preservación de palabras clave del dominio


In [8]:
# Aplicar el preprocesamiento al corpus
df_corpus['texto_procesado'] = df_corpus['documento_original'].apply(preprocess_text_for_classification)

In [14]:
# Generar estadísticas del corpus procesado
print("📊 Estadísticas del corpus procesado:\n")

# Longitud promedio de documentos
longitud_promedio = df_corpus['texto_procesado'].str.len().mean()
print(f"📏 Longitud promedio de documentos: {longitud_promedio:.2f} caracteres")

# Distribución de tipos de documentos
distribucion_tipos = df_corpus['tipo'].value_counts()
print("\n📑 Distribución de tipos de documentos:")
print(distribucion_tipos)

# Palabras únicas por tipo
print("\n🔤 Palabras únicas por tipo de documento:")
for tipo in df_corpus['tipo'].unique():
    palabras_tipo = set(' '.join(df_corpus[df_corpus['tipo'] == tipo]['texto_procesado']).split())
    print(f"\n{tipo}: {len(palabras_tipo)} palabras únicas")

# Guardar el corpus preprocesado en archivo pkl
ruta_guardado = '../data/preprocessed_corpus.pkl'
df_corpus.to_pickle(ruta_guardado)
print(f"\n💾 Corpus preprocesado guardado en: {ruta_guardado}")

# Cargar el corpus preprocesado desde archivo pkl
df_corpus_cargado = pd.read_pickle(ruta_guardado)
print("\n🔍 Cargado desde archivo pkl:")
print(df_corpus_cargado.head())

# Guardar también en formato CSV
ruta_csv = '../data/preprocessed_corpus.csv'
df_corpus.to_csv(ruta_csv, index=False)
print(f"\n💾 Corpus preprocesado guardado en CSV: {ruta_csv}")

# Cargar el corpus desde CSV para verificar
df_corpus_csv = pd.read_csv(ruta_csv)
print("\n🔍 Cargado desde archivo CSV:")
print(df_corpus_csv.head())


📊 Estadísticas del corpus procesado:

📏 Longitud promedio de documentos: 602.15 caracteres

📑 Distribución de tipos de documentos:
tipo
Carta de trabajo        98
Sustento de ingresos    95
Pasaportes              91
Cedulas 2               41
W-9                     19
Name: count, dtype: int64

🔤 Palabras únicas por tipo de documento:

Carta de trabajo: 2977 palabras únicas

Cedulas 2: 381 palabras únicas

Pasaportes: 2001 palabras únicas

Sustento de ingresos: 2339 palabras únicas

W-9: 437 palabras únicas

💾 Corpus preprocesado guardado en: ../data/preprocessed_corpus.pkl

🔍 Cargado desde archivo pkl:
                                  documento_original              tipo  \
0  o!\n\n(O) Tintorería Ecoloca\nN TACO\n\nPara: ...  Carta de trabajo   
1  CICR\n\nCONSTANCIA DE TRABAJO\n\nA quien pueda...  Carta de trabajo   
2  Distribuidora Zoliannys, LLC.\n\nEIN 30-130830...  Carta de trabajo   
3  CONSTANCIA\n\nAla atención de Banco Mercantil ...  Carta de trabajo   
4  Ave. Samuel Le